# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core import Model
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.51.0


In [13]:
import sys
print(sys.version)

3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0]


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl_experiment'
project_folder = './pipeline-project'

experiment=Experiment(ws, experiment_name)

In [3]:
#Source of the data: https://raw.githubusercontent.com/aiplanethub/Datasets/master/HR_comma_sep.csv

# We first check if the data is already loaded inthe Workspace. Otherwise, create it from the file

found = False
key = "HR Dataset"
description_text = "HR Dataset for capstone project"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/aiplanethub/Datasets/master/HR_comma_sep.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
count,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000
mean,0.612834,0.716102,3.803054,201.050337,3.498233,0.144610,0.238083,0.021268
std,0.248631,0.171169,1.232592,49.943099,1.460136,0.351719,0.425924,0.144281
min,0.090000,0.360000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000
25%,0.440000,0.560000,3.000000,156.000000,3.000000,0.000000,0.000000,0.000000
50%,0.640000,0.720000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000
75%,0.820000,0.870000,5.000000,245.000000,4.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000


In [5]:
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [6]:
dataset.take(5).to_pandas_dataframe()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


 ## Cluster creation

In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "cluster-udacity"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster in the provided workspace, so we use it.')
except ComputeTargetException:
    comp_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=3)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, comp_config)

cpu_cluster.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [8]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=cpu_cluster,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="left",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [9]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl_experiment,AutoML_979f9db3-ed3f-4ab4-8b4a-532c929e983a,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [11]:
remote_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
automl_experiment,AutoML_979f9db3-ed3f-4ab4-8b4a-532c929e983a,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.


## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [12]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)
print(remote_run.get_metrics())

Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

Run(Experiment: automl_experiment,
Id: AutoML_979f9db3-ed3f-4ab4-8b4a-532c929e983a_38,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifier', LightGBMClassifier(min_data_in_leaf=20, n_jobs=1, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=None))], verbose=False)), ('1', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('xgboostclassifier', XGBoostC

In [30]:
# Metric in best run
best_run_metrics = best_run.get_metrics()

for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)
    
print('\nAUC_weighted of Best run',best_run_metrics['AUC_weighted'],sep='\n')
print(best_run)

model_name = best_run.properties['model_name']
model_name#Register the model
model = remote_run.register_model(model_name=model_name)
print(model.name, model.id, model.version, sep='\t')

f1_score_micro 0.987665853170634
matthews_correlation 0.9658670595095336
precision_score_macro 0.9867893696581485
accuracy 0.987665853170634
AUC_weighted 0.9947772335754368
recall_score_weighted 0.987665853170634
average_precision_score_macro 0.9941716976070808
f1_score_macro 0.9828775286847286
weighted_accuracy 0.9925376254158368
recall_score_micro 0.987665853170634
recall_score_macro 0.9791087922403822
log_loss 0.050894683742108905
precision_score_weighted 0.9876481411498078
balanced_accuracy 0.9791087922403822
f1_score_weighted 0.987616416761122
precision_score_micro 0.987665853170634
AUC_micro 0.9970711053836195
norm_macro_recall 0.9582175844807642
average_precision_score_micro 0.9969246201941963
average_precision_score_weighted 0.9960623506821248
AUC_macro 0.9947772335754368
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_979f9db3-ed3f-4ab4-8b4a-532c929e983a_38/accuracy_table
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_979f9db3-ed3f-4ab4-8b4a-532c929e983a

In [14]:
# Save the best model
model = remote_run.register_model(model_name='HR-best-automl-model')
print(model.name, model.id, model.version, sep='\t')

HR-best-automl-model	HR-best-automl-model:1	1


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

In [15]:
# Deploy model
from azureml.train.automl.run import AutoMLRun
run_id= 'AutoML_979f9db3-ed3f-4ab4-8b4a-532c929e983a_38' #best run (from the previous cell output)
best_run=AutoMLRun(experiment, run_id)
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl_experiment,AutoML_979f9db3-ed3f-4ab4-8b4a-532c929e983a_38,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [20]:
# Retrive model by name
model = Model(ws, 'HR-best-automl-model')
model

Model(workspace=Workspace.create(name='quick-starts-ws-242066', subscription_id='d7f39349-a66b-446e-aba6-0053c2cf1c11', resource_group='aml-quickstarts-242066'), name=HR-best-automl-model, id=HR-best-automl-model:1, version=1, tags={}, properties={})

In [21]:
env = best_run.get_environment()

script_file = 'score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file)

In [22]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

inference_config = InferenceConfig(entry_script=script_file, environment=env)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                       memory_gb = 1)

deploy_service_name= 'automl-model-deployment'
service = Model.deploy(ws,deploy_service_name,  [model], inference_config, deployment_config)

service.wait_for_deployment(show_output = True)

scoring_uri = service.scoring_uri

print("State: ",service.state)
print("\nScoring URI: ", scoring_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-09-30 17:00:50+00:00 Creating Container Registry if not exists..
2023-09-30 17:10:50+00:00 Registering the environment.
2023-09-30 17:10:51+00:00 Use the existing image.
2023-09-30 17:10:52+00:00 Submitting deployment to compute..
2023-09-30 17:10:57+00:00 Checking the status of deployment automl-model-deployment..
2023-09-30 17:13:42+00:00 Checking the status of inference endpoint automl-model-deployment.
Succeeded
ACI service creation operation finished, operation "Succeeded"
State:  Healthy

Scoring URI:  http://bfd454ae-2063-4b1b-a45b-82f097e5a0e7.southcentralus.azurecontainer.io/score


In [24]:
#Enable application insights
service.update(enable_app_insights=True)

In [26]:
#Test service
%run endpoint_v2.py

ConnectionError: HTTPConnectionPool(host='afcc82ca-85bd-4d21-baf4-835954f56321.westus2.azurecontainer.io', port=80): Max retries exceeded with url: /score (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f919dc8b580>: Failed to establish a new connection: [Errno -2] Name or service not known'))

TODO: In the cell below, print the logs of the web service and delete the service

In [28]:
import json
data_sample = df.sample(2)
y_true = data_sample.pop('left')
sample_json = json.dumps({'data':data_sample.to_dict(orient='records')})
print(sample_json)
output = service.run(sample_json)
print(output)

{"data": [{"satisfaction_level": 0.63, "last_evaluation": 0.66, "number_project": 4, "average_montly_hours": 157, "time_spend_company": 2, "Work_accident": 0, "promotion_last_5years": 0, "Department": "support", "salary": "medium"}, {"satisfaction_level": 0.86, "last_evaluation": 0.85, "number_project": 5, "average_montly_hours": 267, "time_spend_company": 5, "Work_accident": 0, "promotion_last_5years": 0, "Department": "sales", "salary": "medium"}]}
{"result": [0, 0]}


In [23]:
print("\nService Logs:\n",service.get_logs())


Service Logs:
 2023-09-30T17:13:32,906269300+00:00 - rsyslog/run 
2023-09-30T17:13:32,913717500+00:00 - gunicorn/run 
2023-09-30T17:13:32,919853700+00:00 | gunicorn/run | 
2023-09-30T17:13:32,921847900+00:00 | gunicorn/run | ###############################################
2023-09-30T17:13:32,928967600+00:00 | gunicorn/run | AzureML Container Runtime Information
2023-09-30T17:13:32,931057600+00:00 | gunicorn/run | ###############################################
2023-09-30T17:13:32,936710300+00:00 | gunicorn/run | 
2023-09-30T17:13:32,944471600+00:00 | gunicorn/run | 
2023-09-30T17:13:32,947849800+00:00 - nginx/run 
2023-09-30T17:13:32,954470100+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230628.v2
2023-09-30T17:13:32,956130700+00:00 | gunicorn/run | 
2023-09-30T17:13:32,963816600+00:00 | gunicorn/run | 
2023-09-30T17:13:32,970114400+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bi

In [ ]:
#Delete the endpoint
service.delete()

Once all the work has been completed, we proceed to delete the cluster.

In [ ]:
#Delete the cluster
cpu_cluster.delete()

In [ ]:
# Check status of the cluster for deletion 
cpu_cluster

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
